# Automated detection of globular clusters in _Gaia_ DR3

## II: Further cluster analysis

The code below was used to extract more information about the detected clusters such as:
- Radial profiles
- Distances
- Proper motion dispersions

## 1. Radial Profile fitting

We use the [King (1962)](https://ui.adsabs.harvard.edu/abs/1962AJ.....67..471K/abstract) empirical model to map the radial surface density profile of our clusters. The model has 4 fit parameters: 
- $k$ - related to the central surface density; 
- $r_c$: core radius
- $r_t$: tidal radius
- $\eta$ - related to the background density.

We estimated these parameters using the [`emcee`](https://emcee.readthedocs.io/) package. We used the [`corner`](https://corner.readthedocs.io/) package to visualise the corresponding Markov-Chain Monte Carlo samples. The resulting plots/fits are available in the `results/visualisations/radial_profile_fits` directory.

In [1]:
from analysis import radial_mle, corner_plot
import numpy as np

file_path = '../results/clus_mem_lists/Eridanus.ecsv'
r_lower, r_upper = 0, 4
priors = np.array([[0,75], [0.2,0.5], [0,5], [-10,10]])
p0 = [50,0.25,1,0.1]
samples = radial_mle(file_path, r_lower, r_upper, priors, p0)

corner_plot(file_path, samples, r_lower, r_upper)

100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2183.76it/s]


Best-fit parameters: k=58.825, r_c=0.351, r_t=2.722, eta=0.023
Concentration parameter: c = 0.889 +/- 0.680


In [2]:
from analysis import radial_mle, corner_plot
import numpy as np

file_path = '../results/clus_mem_lists/NGC 5904.ecsv'
r_lower, r_upper = 4,20
priors = np.array([[0,350], [0,20], [0,25], [0,15]])
p0 = [200,1,5,2]
samples = radial_mle(file_path, r_lower, r_upper, priors, p0)
    
corner_plot(file_path, samples, r_lower, r_upper)

100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2069.54it/s]


Best-fit parameters: k=242.584, r_c=3.765, r_t=19.798, eta=0.335
Concentration parameter: c = 0.721 +/- 0.297


In [3]:
from analysis import radial_mle, corner_plot
import numpy as np

file_path = '../results/clus_mem_lists/Pal 5.ecsv'
r_lower, r_upper = 0, 8
priors = np.array([[0,100], [3,6], [0,10], [-10,10]])
p0 = [30,4,4,0]
samples = radial_mle(file_path, r_lower, r_upper, priors, p0)

corner_plot(file_path, samples, r_lower, r_upper)


100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2065.01it/s]


Best-fit parameters: k=43.766, r_c=4.755, r_t=6.763, eta=0.179
Concentration parameter: c = 0.153 +/- 0.221


## 2. Distances:

We use the `emcee` package again to estimate cluster distances based on two approaches:

- Parallax
- CMD fitting (to be added later)

Details of the fitting procedures can be found in [Baloyi et al. (2025)](link to be inserted).

In [4]:
import os
from analysis import plx_dist

clus_mem_list_dir = '../results/clus_mem_lists/'
for file in os.listdir(clus_mem_list_dir):
    clus_name, dist, err_lo, err_hi = plx_dist(f'{clus_mem_list_dir}{file}')
    print(f'{clus_name}:\nD = {dist:.3f} (-{err_lo:.3f} / + {err_hi:.3f}) kpc')

100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2301.41it/s]


Pal 5:
D = 25.173 (-5.000 / + 7.856) kpc


100%|███████████████████████████████████| 10000/10000 [00:06<00:00, 1546.10it/s]


NGC 5904:
D = 7.426 (-0.087 / + 0.086) kpc


100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2426.85it/s]


Eridanus:
D = 6.774 (-1.430 / + 2.683) kpc


In comparison to literature, e.g. [Baumgardt and Vasiliev (2021)](https://ui.adsabs.harvard.edu/abs/2021MNRAS.505.5957B/abstract), only Eridanus' distance above is WAY off, with $D_{\rm Eri, lit} = {84.684}_{-2.838}^{+2.936}$ kpc. For sources at large distances, parallax ceases to be a reliable measure of distance.

## 3. Proper motion dispersion:

We re-use `emcee` to calculate observed total proper motion (PM) dispersion of a given cluster. If the distance to the cluster is known, it is a straightforward process to calculate the tangential velocity dispersion from the PM dispersion - i.e. by using $v \ [\mathrm{km/s}] = 4.74 \mu \ [\mathrm{mas/yr}] \ D \ [\mathrm{kpc}]$.

In [5]:
import os
from analysis import pm_mle

clus_mem_list_dir = '../results/clus_mem_lists/'
for file in os.listdir(clus_mem_list_dir):
    clus_name, disp, disp_err_lo, disp_err_hi = pm_mle(f'{clus_mem_list_dir}{file}')
    print(f'{clus_name}:\nPM Dispersion = {disp:.3f} (-{disp_err_lo:.3f} / + {disp_err_hi:.3f}) mas/yr')


100%|█████████████████████████████████████| 5000/5000 [00:01<00:00, 2962.31it/s]


Pal 5:
PM Dispersion = 0.012 (-0.008 / + 0.013) mas/yr


100%|█████████████████████████████████████| 5000/5000 [00:03<00:00, 1560.95it/s]


NGC 5904:
PM Dispersion = 0.072 (-0.003 / + 0.003) mas/yr


100%|█████████████████████████████████████| 5000/5000 [00:01<00:00, 3093.07it/s]


Eridanus:
PM Dispersion = 0.294 (-0.054 / + 0.062) mas/yr
